In [ ]:
inpath = ""
outpath_nexus = None
outpath_shifts = None
alignment_section = "all"
alignment_band = 15

## Usage 

This notebook determines the horizontal shifts of stacked XRF tomography data and saves the alignment projections (`outpath_nexus`) alongside the shifts (`output_shifts`). This rough initial alignment is based on finding the centre of mass for each projection.

### Parameters

`inpath` : str
The relative path to a stacked XRF tomography nexus file containing projection data.

`outpath_nexus` : str
The relative path to the output nexus containing the initially aligned projection data.

`outpath_shifts` : str
The relative path to the .txt file with the shifts that were used to align the data.

`alignment_section`: str
The vertical section of slices to be used for alignment, choose from "all", "top", "middle" or "bottom". Default is "all".

`alignment_band` : int
The vertical band of slices to be used for alignment in units of pixels, default is 15.

### Dependencies

- Numpy
- SciPy
- h5py
- matplotlib
- imageio


In [ ]:
import h5py
import imageio
import pathlib
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

## Create folder for assets

In [ ]:
assets_folder = pathlib.Path("./_assets")
assets_folder.mkdir(parents=True, exist_ok=True)

## Check parameters

In [ ]:
assert inpath is not None, "Need to provide Nexus NXtomo file."
assert inpath.endswith(".nxs"), f"The provided input file {inpath} needs to end with .nxs"
inpath_nexus = pathlib.Path(inpath)

if outpath_nexus is None:
    outpath_nexus = inpath_nexus.stem + "_aligned_initial.nxs"
if outpath_shifts is None:
    outpath_shifts = inpath_nexus.stem + "_shifts.txt"

## Loading data from Nexus NXtomo file

In [ ]:
with h5py.File(inpath, "r") as f:
    assert "entry" in f and \
           "definition" in f["entry"] and \
           f["entry/definition"][()] == b"NXtomo", \
           f"{inpath} is not a Nexus file of type NXtomo -> use nxstacker to generate stacked XRF projections."
    tomo = np.nan_to_num(f["entry/data/data"][:])

## Pick whether to align to all, middle, top or bottom slices of data
This can be useful if pin is strong signal and want to crop, for example

You can adjust band = +/- this num pixels from top, middle or bottm
Set section to all, top, middle or bottom to select option  
Generally more data the better

In [ ]:
sizey, sizex = tomo.shape[1:]
if alignment_section == "top":
    b_low = 0
    b_high = min(band,sizey)
elif alignment_section=="middle":
    b_low = max(int(sizey/2 - alginment_band/2),0)
    b_high = min(int(sizey/2 + alignment_band/2),sizey)
elif alignment_section == "bottom":
    b_low = max(int(sizey - alignment_band),0)
    b_high = sizey
else:
    b_low = 0
    b_high = sizey
to_align = tomo[:,b_low:b_high].copy()

##  Centre of mass alignment

In [ ]:
def com_line(dd):
    summ=0.0
    for i in range(len(dd)):
        summ=summ+i*dd[i]
    return summ/np.sum(dd)

In [ ]:
shifts = []
for sig1 in to_align:
    ss=sig1.sum(axis=1)
    xs=com_line(ss)
    ss=sig1.sum(axis=0)
    ys=com_line(ss)
    shifts.append([xs,ys])
shifts = np.array(shifts)
ccx=int(to_align.shape[2]/2)
ccy=int(to_align.shape[1]/2)
shifts=shifts-np.array([ccy,ccx])

In [ ]:
to_align = np.nan_to_num(np.array([ndi.shift(to_align[i], -shifts[i], order=1) for i in range(len(to_align))]))

In [ ]:
maxv = np.max(to_align)
minv = np.min(to_align)
output_file = assets_folder / f"{inpath_nexus.stem}_align_slice.gif"
image = []
for i in range(len(to_align)):
    data = to_align[i] - minv
    data = data/(maxv-minv)
    data = 255 * data # Now scale by 255
    img = data.astype(np.uint8)
    image.append(img)
imageio.mimsave(output_file, image)

In [ ]:
from IPython.display import Image
print("File",output_file)
display(Image(output_file,width=512))

# Check what this looks like on whole data 

In [ ]:
to_align = np.nan_to_num(np.array([ndi.shift(tomo[i], -shifts[i], order=1) for i in range(len(tomo))]))

In [ ]:
maxv = np.max(to_align)
minv = np.min(to_align)
output_file = assets_folder / "align_stack.gif"
image = []
for i in range(len(to_align)):
    data = to_align[i] - minv
    data = data/(maxv-minv)
    data = 255 * data # Now scale by 255
    img = data.astype(np.uint8)
    image.append(img)
imageio.mimsave(output_file, image)

In [ ]:
from IPython.display import Image
print("File",output_file)
display(Image(output_file,width=512))

## Saving the aligned stack into Nexus NXtomo file

In [ ]:
with h5py.File(inpath, "r") as fin:
    with h5py.File(outpath_nexus, "w") as fout:
        fin.copy(fin["entry"], fout, "entry")
        fout["entry/data/data"][:] = to_align

## Saving the shifts into a TXT file

In [ ]:
np.savetxt(outpath_shifts, shifts, delimiter=" ")